In [7]:
import numpy as np
from scipy.stats import norm


def generate_data(n, pk1, a1, a2): 
    k_states = np.random.choice([1, 2], size=n, p=[pk1, 1 - pk1])
    means = np.where(k_states == 1, a1, a2)
    data = np.random.normal(loc=means, scale=1)
    return data, k_states   #Генерація випадкових величин за умовою задачі

def expectation_maximization(data, max_iterations=1000, tol=0.001): #FАлгоритм самонавчання 
    n = len(data)
    pk1 = 0.5
    pk2 = 1 - pk1
    a1 = 1
    a2 = 2

    for iteration in range(max_iterations):
        pdf_k1 = norm.pdf(data, loc=a1, scale=1)
        pdf_k2 = norm.pdf(data, loc=a2, scale=1)
        gamma_k1 = pk1 * pdf_k1 / (pk1 * pdf_k1 + pk2 * pdf_k2)
        gamma_k2 = pk2 * pdf_k2 / (pk1 * pdf_k1 + pk2 * pdf_k2)

        pk1_new = np.mean(gamma_k1)
        pk2_new = np.mean(gamma_k2)
        
        f1 = norm.pdf(data, loc=0, scale=1)
        f2 = norm.pdf(data, loc=1, scale=1)
        f3 = norm.pdf(data, loc=2, scale=1)
        f4 = norm.pdf(data, loc=3, scale=1)
        
        
        func1 = np.sum(gamma_k1 *np.log(f1))
        func2 = np.sum(gamma_k1 *np.log(f2))
        if (func1 > func2): 
            a1_new = 0
        else: 
            a1_new = 1
            
        func3 = np.sum(gamma_k2 *np.log(f3))
        func4 = np.sum(gamma_k2 *np.log(f4))
        if (func3 > func4): 
            a2_new = 2
        else: 
            a2_new = 3

        if np.abs(pk1_new - pk1) < tol and  a1_new - a1 == 0 and \
           np.abs(pk2_new - pk2) < tol and a2_new - a2 == 0:
            break

        pk1, pk2, a1, a2 = pk1_new, pk2_new, a1_new, a2_new

    return pk1, pk2, a1, a2

np.random.seed(42) 
n = 100
pk1_true = 1/3
a1_true = 0
a2_true = 3

data, true_k_states = generate_data(n, pk1_true, a1_true, a2_true)

p_k1_init = 0.5
a1_init = 1
a2_init = 2

pk1_hat, pk2_hat, a1_hat, a2_hat = expectation_maximization(data)

print("Справжні значення:")
print("P(k=1) =", pk1_true, "a1 =", a1_true)
print("P(k=2) =", 1 - pk1_true, "a2 =", a2_true)

print("Оцінені значення:")
print("P(k=1) =", pk1_hat, "a1 =", a1_hat)
print("P(k=2) =", pk2_hat, "a2 =", a2_hat)

Справжні значення:
P(k=1) = 0.3333333333333333 a1 = 0
P(k=2) = 0.6666666666666667 a2 = 3
Оцінені значення:
P(k=1) = 0.5341101250146805 a1 = 1
P(k=2) = 0.4658898749853195 a2 = 3
